In [1]:
from keras import models 
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, Convolution2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import glob
import h5py
import os
import matplotlib.pyplot as plt
from google.colab import drive
import numpy as np
from tqdm import tqdm


Using TensorFlow backend.


In [2]:
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls
os.chdir('gdrive/My Drive/My_AI_proj/CERN_data/')
!pwd

gdrive	sample_data
/content/gdrive/My Drive/My_AI_proj/CERN_data


In [0]:

photon='SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
electron='SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'


In [0]:
def create_model():
    model=Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(32,32,2), activation='relu'))
    # print(model.shape)
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    # print(model.shape)
    model.add(MaxPooling2D(pool_size=(2,2)))
    # print(model.shape)
    model.add(Flatten())
    
    # model.add(Dense(units=3776, activation='relu'))
    model.add(Dense(units=1500, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(units=500, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=100, activation='relu'))

    optimizer=Adam(lr=1e-1)    
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
def input_target(photon, electron):
  X_input=[]
  electron_image_data=h5py.File(electron,'r')['X'].value
  for i in tqdm(range(len(electron_image_data))):
    image=electron_image_data[i]
    X_input.append(image)
  del(electron_image_data)
  photon_image_data=h5py.File(photon, 'r')['X'].value
  for i in tqdm(range(len(photon_image_data))):
    image=photon_image_data[i]
    X_input.append(image)
  y_photon=[1]*249000
  y_electron=[0]*249000
  y_target=np.array(y_photon+y_electron)
  return np.asarray(X_input), np.asarray(y_target)


In [7]:
X_input,y_target=input_target(photon,electron)

100%|██████████| 249000/249000 [00:00<00:00, 1672753.05it/s]


In [8]:
print(X_input.shape, y_target.shape)

(498000, 32, 32, 2) (498000,)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_input,y_target, test_size=0.2,random_state=23)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(298800, 32, 32, 2) (199200, 32, 32, 2) (298800,) (199200,)


In [13]:
model=create_model()
model.summary()
history=model.fit(x=X_train, y=y_train, batch_size=100, epochs=40, verbose=1, callbacks=None,validation_data=(X_test,y_test), shuffle=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 2..., activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  """


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        608       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1500)              9409500   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1500)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)              

In [0]:
 model.save('l:0.3892 a:0.8232 vl:0.6803 va: 0.7110.hdf5')